# Automated pipeline to run BFE corrections in stage 1
Takes an _uncal_ file and can apply either the 'grid' correction (directly applying ramp solutions from the closest pointing), or the 'interpolated' correction (gets ramp solutions based on an interpolated function using the offset from PSF centre). Requires a _rate_ file to centroid on. Currently only applicable to channels 1 and 2.

In [1]:
from jwst.pipeline import Detector1Pipeline
from jwst import datamodels
from jwst.datamodels import dqflags
from jwst.reset import ResetStep
from jwst.saturation import SaturationStep
from jwst.linearity import LinearityStep

# JWST pipeline utilities
from jwst import datamodels # JWST datamodels
from os import mkdir
from os.path import isdir

import shutil
import os
from os import path
from os.path import isfile
from os import listdir
import sys
import getopt
from astropy.io import fits
import numpy as np
import datetime
import glob as glob
import core.linearity_coeff_ref_files as lin_ref


from IPython.core.display import display, HTML

from crds import client
%env CRDS_PATH crds_cache/
print(os.environ["CRDS_PATH"])
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

os.environ["CRDS_CONTEXT"] = "jwst_1235.pmap"

2025-03-27 10:09:08,540 - stpipe - WARNING - /var/folders/fw/_g4g808x2fxcgl7stjth5mxh0000gn/T/ipykernel_2025/1916562549.py:27: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML



env: CRDS_PATH=crds_cache/
crds_cache/


In [2]:
inputDir = "../DATA/twhya/uncal/"#'../DATA/GQLup/sci_uncal/' ## directory of files to reduce
rateDir = "../DATA/twhya/rate/"#'../DATA/GQLup/sci/' ## directory of rate files to centroid on (same file but with '_rate')
outDir = "../DATA/twhya/sci_interp_bfe/"#'../DATA/GQLup/sci_BFE_grid' ## directory where reduced data will be saved. Directory must already exist.
refDir = './data/'
file_list = [aa.split("/")[-1] for aa in glob.glob(inputDir+"/*_uncal.fits")]

linFile = 'interp' #'grid' or 'interp'
saveRamp = False # set to True if you want to check the calibrated ramps
distVers = 'flt7' # version of distortion files

if not isdir(outDir): mkdir(outDir)

In [3]:
def run_pipeline(list_of_files, input_dir, output_dir, rate_dir, lin_file, step=1, ramp_save = False):

    for i in range(len(list_of_files)):
        
        file=list_of_files[i]
        print(list_of_files[i])
        
        if step==1:
            
            filesplit = file.split('_')
            ratefile = filesplit[0] + '_' + filesplit[1] + '_' + filesplit[2] + '_' + filesplit[3] + '_rate.fits'
            
            rate = fits.open(rate_dir+ratefile)
            detector = rate[0].header['DETECTOR']      
            chan = rate[0].header['CHANNEL']            
            print(detector)

            if detector != 'MIRIFULONG':

                if lin_file == 'interp':

                    lin_ref.gen_custom_linearity(rate,refDir,num=1,dist_ver='flt7')
                    linearity_file = './custom_linearity_ref_{}.fits'.format(1)
                elif lin_file == 'grid':
                    linearity_file = lin_ref.find_nearest_grid(rate,refDir,dist_ver='flt7')
                
                pipe = Detector1Pipeline()

                pipe.save_calibrated_ramp = ramp_save
    
                pipe.linearity.skip = False
                pipe.group_scale.skip = True
                pipe.dq_init.skip = False
                pipe.superbias.skip = False
                pipe.gain_scale.skip = True
                # pipe.emicorr.skip = False
                pipe.saturation.skip = False
                pipe.ipc.skip = True ##bad!
                pipe.firstframe.skip = False
                pipe.lastframe.skip = False
                pipe.reset.skip = False
                pipe.saturation.n_pix_grow_sat = 0
                #pipe.charge_migration.skip = True
                pipe.linearity.override_linearity = './'+linearity_file
                pipe.rscd.skip = False
                pipe.dark_current.skip = False
                pipe.refpix.skip = False
    
                pipe.save_results = True
                pipe.output_dir = output_dir

                result = pipe.run(input_dir+file)
            elif detector == 'MIRIFULONG':
                shutil.copyfile(rateDir+file.replace("uncal","rate"), outDir+file.replace("uncal","rate"))



In [4]:
# file_list = [aa for aa in glob.glob(rateDir+"/*masked_rate.fits")]
# print(file_list)
# for ii in file_list:
#     os.rename(ii, ii.replace("_masked",""))


In [5]:
run_pipeline(file_list, inputDir, outDir, rateDir, lin_file = linFile, ramp_save = saveRamp)

jw01282020001_03102_00001_mirifushort_uncal.fits
MIRIFUSHORT
STEP 1: Rough centroiding
Slice 14 has the largest summed flux
Source position: beta = 0.53arcsec, alpha = -0.11arcsec 

STEP 2: 1D Gaussian fit
[Along-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
[Across-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]

STEP 3: 2D Gaussian fit
The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 999, 10

2025-03-27 10:15:37,293 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-03-27 10:15:37,294 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-03-27 10:15:37,295 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-03-27 10:15:37,295 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2025-03-27 10:15:37,296 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2025-03-27 10:15:37,297 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2025-03-27 10:15:37,297 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance created.
2025-03-27 10:15:37,298 - stpipe.Detector1Pipeline.refpix - INFO - RefPixStep instance created.
2025-03-27 10:15:37,299 - stpipe.Detector1Pipeline.rscd - INFO - RscdStep instance created.
2025-03-27 10:15:37,299 - stpipe.Detector1Pipeline.firstframe - INFO - FirstFrameStep instance created.
2025-03-27 10:15

jw01282020001_03104_00002_mirifulong_uncal.fits
MIRIFULONG
jw01282020001_03106_00001_mirifulong_uncal.fits
MIRIFULONG
jw01282020001_03104_00003_mirifushort_uncal.fits
MIRIFUSHORT


2025-03-27 10:16:15,824 - stpipe - WARNING - /Users/sand-jrd/PSF-SUBTRACTION-MINDS/STEP1-BFECORR/core/linearity_coeff_ref_files.py:52: ResourceWarning: unclosed file <_io.BufferedReader name='./data/MIRI_FM_MIRIFUSHORT_12MEDIUM_DISTORTION_flt7.fits'>
  d2cMaps[b] = d2cMapping(b[0]+subbandl,dist_dir,slice_transmission='10pc',fileversion = "flt7")

2025-03-27 10:16:16,058 - stpipe - WARNING - /opt/miniconda3/envs/minds_env/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',



STEP 1: Rough centroiding
Slice 13 has the largest summed flux
Source position: beta = 0.35arcsec, alpha = 0.1arcsec 

STEP 2: 1D Gaussian fit
[Along-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
[Across-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 339, 340, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]

STEP 3: 2D Gaussian fit


...

2025-03-27 11:09:49,301 - stpipe - WARNING - /opt/miniconda3/envs/minds_env/lib/python3.11/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',



[Along-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 52, 56, 61, 981, 982, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
[Across-slice fit] The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 56, 61, 981, 982, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]

STEP 3: 2D Gaussian fit


2025-03-27 11:10:04,067 - stpipe - WARNING - /Users/sand-jrd/PSF-SUBTRACTION-MINDS/STEP1-BFECORR/core/linearity_coeff_ref_files.py:68: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  coord=np.where(d2cMaps[b]['sliceMap'][ypos[512],xpos[512]] == sliceID+slice_off)



The following bins failed to converge:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 52, 56, 61, 981, 982, 984, 985, 986, 987, 988, 989, 990, 991, 992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023]
0.11172492868357153 -0.028131399039262206


2025-03-27 11:10:39,640 - stpipe - WARNING - /var/folders/fw/_g4g808x2fxcgl7stjth5mxh0000gn/T/ipykernel_2025/3612107258.py:22: ResourceWarning: unclosed file <_io.BufferedReader name='./crds_cache/references/jwst/miri/jwst_miri_linearity_0030.fits'>
  lin_ref.gen_custom_linearity(rate,refDir,num=1,dist_ver='flt7')

2025-03-27 11:10:39,702 - stpipe.Detector1Pipeline - INFO - Detector1Pipeline instance created.
2025-03-27 11:10:39,703 - stpipe.Detector1Pipeline.group_scale - INFO - GroupScaleStep instance created.
2025-03-27 11:10:39,703 - stpipe.Detector1Pipeline.dq_init - INFO - DQInitStep instance created.
2025-03-27 11:10:39,704 - stpipe.Detector1Pipeline.emicorr - INFO - EmiCorrStep instance created.
2025-03-27 11:10:39,705 - stpipe.Detector1Pipeline.saturation - INFO - SaturationStep instance created.
2025-03-27 11:10:39,705 - stpipe.Detector1Pipeline.ipc - INFO - IPCStep instance created.
2025-03-27 11:10:39,706 - stpipe.Detector1Pipeline.superbias - INFO - SuperBiasStep instance 